# Gene linker pipeline for "Chromatin looping links target genes with genetic risk loci for dermatological traits"

preprint available at https://www.biorxiv.org/content/10.1101/2020.03.05.973271v2

import all libraries needed for analysis.
Suggested using the Conda environment.

In [24]:
import pandas as pd
import numpy as np
import os
import scipy
import pybedtools as pbed 
import helpers.gene_link
import subprocess as sub
import math
import statistics
import itertools
from concurrent.futures import ProcessPoolExecutor
os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

In [49]:
# settings. leave as it is for same results as paper

# filter genes by RNA-seq?
RNA_seq_filter = True

# filter SNPs by active peak?
SNP_filter_peak_OE = False
SNP_filter_peak_overlapping = True

# add overlapping genes to results?
Append_overlapping = True

# do you have TADs?
report_TADs = True

# output folder
output_folder = "../example_results/"

Declare all GWAS files

format required:

chr  start  end(start+1)  score  snpid_LOCI

In [60]:
# location of GWAS annotation
GWAS_loc = "../datasets/gwas"

GWAS_files = ["bowes_stuart.ld.hg38.bed","duffy.ld.hg38.bed","eczema_no_hla.ld.hg38.bed","elena.ld.hg38.bed","tsoi2017_LD_0.8_hg38.bed"]

snp_names = ["PsA", "Ps", "at_derm", "melanoma", "sclerosis"]

snp_dfs = {key:pd.read_csv(os.path.join(GWAS_loc,value), sep="\t",header=None) for (key,value) in zip(snp_names,GWAS_files)}
for x in snp_dfs.values():
    x.columns = "chr start end name score".split()
    x["loci"] = x["name"].str.split("_").str[-1]

Declare all loops and peaks files needed.

Use processed files with HiC-pro and then use FitHiChIP and HiChIP-peaks to identify loops and peaks.

We use the stringent, merged interactions.

In [61]:
# location of the loops files
loops_loc = "../datasets/loops"

# name your files with FULL_NAME and if you have replicates use FULL_NAME_N with N = 1,2... (currently max 6 but you can change code down)
# avoid naming samples with a name that is a substring of another sample (e.g. T_cell and T_cell_th17)
names = ["hacat_stim_1","hacat_stim_2","hacat_unstim_1","hacat_unstim_2","myla_1","myla_2","naive_T_1","naive_T_2","GM12878","hacat_stim","hacat_unstim","myla","naive_T",]
# give the names of the full(combined samples)
full_names = ["GM12878","hacat_stim","hacat_unstim","myla","naive_T"]

# give list of files IN THE SAME ORDER as the names provided above
files = ["HaCat_27ac_st_rep1_helen_stringent_merged_interactions_Q0.01.bed",
         "HaCat_27ac_st_rep2_helen_stringent_merged_interactions_Q0.01.bed",
         "HaCat_27ac_unst_rep1_helen_stringent_merged_interactions_Q0.01.bed",
         "HaCat_27ac_unst_rep2_helen_stringent_merged_interactions_Q0.01.bed",
         "MyLa_27ac_rep1_helen_stringent_merged_interactions_Q0.01.bed",
         "MyLa_27ac_rep2_helen_stringent_merged_interactions_Q0.01.bed",
         "NaiveT_27ac_B2_mumbach_stringent_merged_interactions_Q0.01.bed",
         "NaiveT_27ac_B3_mumbach_stringent_merged_interactions_Q0.01.bed",
         "GM12878_H3K27ac_mumbach_combined_stringent_merged_interactions_Q0.01.bed",
        "HaCaT_st_27ac_helen_combined_stringent_merged_interactions_Q0.01.bed",
        "HaCaT_unst_27ac_helen_combined_stringent_merged_interactions_Q0.01.bed",
        "MyLa_27ac_helen_combined_stringent_merged_interactions_Q0.01.bed",
        "Naive_Tcells_H3K27ac_mumbach_combined_stringent_merged_interactions_Q0.01.bed",]

dict_loops = {key:pd.read_csv(os.path.join(loops_loc,value), sep="\t") for (key,value) in zip(names,files)}

In [62]:
# load all peaks datasets
# position of the peaks files
peaks_loc = "../datasets/peaks"

# provide the list of file names in the same order as above
peaks_fn = ["HaCat_27ac_st_rep1_helenpeaks.bed", "HaCat_27ac_st_rep2_helenpeaks.bed", "HaCat_27ac_unst_rep1_helenpeaks.bed", "HaCat_27ac_unst_rep2_helenpeaks.bed",
            "MyLa_27ac_rep1_helenpeaks.bed", "MyLa_27ac_rep2_helenpeaks.bed", "NaiveT_27ac_B2_mumbachpeaks.bed", "NaiveT_27ac_B3_mumbachpeaks.bed",
            "GM12878_H3K27ac_mumbach_combinedpeaks.bed","HaCaT_st_27ac_helen_combinedpeaks.bed","HaCaT_unst_27ac_helen_combinedpeaks.bed","MyLa_27ac_helen_combinedpeaks.bed",
            "Naive_Tcells_H3K27ac_mumbach_combinedpeaks.bed"]

dict_peaks = {key:pd.read_csv(os.path.join(peaks_loc,value), sep="\t", header=None) for (key,value) in zip(names,peaks_fn)}

Declare TAD files.

TADs in this script have been called using OnTAD.

In [72]:
# for which samples are TADs available?
tad_samples = ["hacat_stim","hacat_unstim","myla"]

tads_loc = "../datasets/TADs"
tad_files = ["OnTAD_hacat_stim_all_chr_sorted.bed.gz",
        "OnTAD_hacat_unstim_all_chr_sorted.bed.gz",
        "OnTAD_myla_all_chr_sorted.bed.gz"]
# dict to map color of the tad to the tad level reported by ontad
color_map = {'127,201,127':2, '190,174,212':3, '253,192,134':4, '255,0,0':5, '56,108,176':1}

tads_dfs_merged = {key:pd.read_csv(os.path.join(tads_loc,x) , sep="\t", header=None, converters={8:lambda x: color_map[x]}) for (key,x) in zip(tad_samples,tad_files)}


Declare RNA-seq files.

Script only links genes that are transcribed in the cell type

In [64]:
rna_seq_location = "../datasets/RNA-seq/Expression_TPM.csv"
expressed_genes = pd.read_csv(rna_seq_location, index_col = 0)

## Start of actual script
All code belove has been reassembled to work with most conditions above. If you need help please contact me.

In [65]:
# load annotation for transcripts
try:
    gtf_annotation_df = pd.read_pickle("../datasets/references/gencode_gtf.pickle")
except FileNotFoundError:
    print("could not find pickled version of gencode reference, recreating new (slow)")
    import helpers.gtf_reader
    gtf_annotation = "../datasets/references/gencode.v29.primary_assembly.annotation.gtf.gz"
    gtf_annotation_df = helpers.gtf_reader.gtf_dataframe(gtf_annotation)

In [66]:
# get transcripts, reformat gene_id and identify start site.
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)

### link genes 

In [67]:
# link all genes

def _cf_helper(x):
    return helpers.gene_link.link_genes(x, dict_loops,dict_peaks,gtf_transcripts, SNP_filter_peak_OE, SNP_filter_peak_overlapping)

with ProcessPoolExecutor(max_workers = 8) as executor:
    result = list(executor.map(_cf_helper, snp_dfs.values()))
    OtherEnd_genes = {k:v[0] for k,v in zip(snp_dfs, result)}
    Overlapping_genes = {k:v[1] for k,v in zip(snp_dfs, result)}


# Unparellized version if you need it
# OtherEnd_genes = {}
# Overlapping_genes = {}

# for name, x in snp_dfs.items():
#     OtherEnd_genes[name], Overlapping_genes[name] = helpers.gene_link.link_genes(x,dict_loops,dict_peaks,gtf_transcripts, SNP_filter_peak_OE, SNP_filter_peak_overlapping)


In [68]:
def join_set(x):
    # helper function to link snps in a set for pandas
    l = []
    for i in x:
        if not pd.isna(i):
            l.extend(i)
    if len(set(l)) == 0:
        return np.nan
    else:
        return set(l)

# mapping between RNA-seq and samples
sample_cond = {}
for sample in expressed_genes.columns:
    if sample in names:
        sample_cond[sample] = sample
        for i in names:
            if sample in i:
                sample_cond[i] = sample
    
# group originating snps together and filter genes based on expression
agg_act = {**dict.fromkeys(names, 'min'),**dict.fromkeys(["transcript_" + x for x in names],join_set),**dict.fromkeys(["linked_SNP_" + x for x in names],join_set)}
genes_dfs = {}
for snp_name in Overlapping_genes:
    if RNA_seq_filter:
        for sample in names:
            OtherEnd_genes[snp_name].loc[OtherEnd_genes[snp_name]["gene_id"].isin(expressed_genes[expressed_genes[sample_cond[sample]] < 1].index),[sample,"transcript_" + sample, "linked_SNP_" + sample]] = np.nan
            Overlapping_genes[snp_name].loc[Overlapping_genes[snp_name]["gene_id"].isin(expressed_genes[expressed_genes[sample_cond[sample]] < 1].index),[sample,"transcript_" + sample, "linked_SNP_" + sample]] = np.nan
    OtherEnd_genes[snp_name] = OtherEnd_genes[snp_name].dropna(how="all",subset=names)
    Overlapping_genes[snp_name] = Overlapping_genes[snp_name].dropna(how="all",subset=names)
    if Append_overlapping:
        genes_dfs[snp_name] = pd.concat((OtherEnd_genes[snp_name],Overlapping_genes[snp_name]),sort=False)
    else:
        genes_dfs[snp_name] = OtherEnd_genes[snp_name]
    genes_dfs[snp_name] = genes_dfs[snp_name].groupby("gene_id",as_index = False).agg(agg_act)
    



In [100]:
#save processed intermediate data
for SNP, df_OE in OtherEnd_genes.items():
    df_OE.merge(gtf_transcripts[["gene_id", "gene_name"]].drop_duplicates()).to_csv(
    os.path.join(output_folder, "genes_OE_" + SNP + ".csv"))
    df_OE.merge(gtf_transcripts[["gene_id", "gene_name"]].drop_duplicates()).to_pickle(
    os.path.join(output_folder, "genes_OE_" + SNP + ".pkl"))
for SNP, df_Over in Overlapping_genes.items():
    df_Over.merge(gtf_transcripts[["gene_id", "gene_name"]].drop_duplicates()).to_csv(
    os.path.join(output_folder, "genes_overlapping_" + SNP + ".csv"))
    df_Over.merge(gtf_transcripts[["gene_id", "gene_name"]].drop_duplicates()).to_pickle(
    os.path.join(output_folder, "genes_overlapping_" + SNP + ".pkl"))
for SNP, value in genes_dfs.items():
    value[["gene_id"] + names].merge(gtf_transcripts[["gene_id", "gene_name"]].drop_duplicates()).to_csv(
    os.path.join(output_folder, "genes_linked_" + SNP + ".csv"))
    value[["gene_id"] + names].merge(gtf_transcripts[["gene_id", "gene_name"]].drop_duplicates()).to_pickle(
    os.path.join(output_folder, "genes_linked_" + SNP + ".pkl"))
for SNP in genes_dfs:
    genes_dfs[SNP] = genes_dfs[SNP].merge(gtf_transcripts[["gene_id", "gene_name"]].drop_duplicates())
    genes_dfs[SNP].to_csv(os.path.join(output_folder, "genes_linked_with_info_" + SNP + ".csv"))
    genes_dfs[SNP].to_pickle(os.path.join(output_folder, "genes_linked_with_info_" + SNP + ".pkl"))


### link TADs

In [101]:
def identify_tad_level(genes_df, snp_df):
    def _get_TAD(gene_coord, loci_start, loci_end, cond):
        cond = cond.rstrip("_1")
        cond = cond.rstrip("_2")
        cond = cond.rstrip("_3")
        cond = cond.rstrip("_4")
        cond = cond.rstrip("_5")
        cond = cond.rstrip("_6")
        tad_df = tads_dfs_merged[cond]
        tads_gene = tad_df[(tad_df[0] == gene_coord[0]) & (tad_df[1] <= int(gene_coord[1]) - 40000) & (tad_df[2] >= int(gene_coord[2]) + 40000)]
        tads = tads_gene[(tads_gene[1] < loci_end + 40000) & (tads_gene[2] > loci_start - 40000)]
        if len(tads.index) == 0:
            if len(tads_gene.index) == 0:
                return np.nan
            return 0
        return tads.loc[tads[8].idxmax()][8]
    
    def _get_gene_coord(gene):
        transcripts = gtf_transcripts[gtf_transcripts["gene_id"] == gene]
        return [transcripts["seqname"].iloc[0], transcripts["start"].min(), transcripts["end"].max()]
    
    def _get_loci_coords(row_gene):
        gene_coord = _get_gene_coord(row_gene["gene_id"])
        locis = []
        for i in names:
            # for each cell type, if it has been linked
            if type(row_gene["linked_SNP_" + i]) == set:
                loci = set([x.split("_")[-1] for x in row_gene["linked_SNP_" + i]])
                locis.extend(list(loci))
                if i in conditions_with_TADs:
                    row_gene["TAD_level_" + i] = {}
                    for j in range(len(loci)):
                        t = snp_df[snp_df["loci"] == list(loci)[j]]
                        loci_start = t["start"].min()
                        loci_end = t["end"].max()
                        tad_level = _get_TAD(gene_coord, loci_start, loci_end, i)
                        row_gene["TAD_level_" + i][list(loci)[j]] = tad_level
        row_gene["locis"] = list(loci)
        return row_gene
    
    def _get_TAD_level_final(row, cond, loci):
        cols = [x for x in row.index if "TAD_level_" + cond in x]
        vals = []
        for i in cols:
            if (type(row[i]) != float) and (loci in row[i].keys()):
                vals.append(row[i][loci])
        if vals != []:
            return min(vals)
        else:
            return np.nan
    
    conditions_with_TADs = [x for x in names if any(s in x for s in tad_samples)]
    
    coords = genes_df.apply(_get_loci_coords, axis = 1)
    df = pd.DataFrame()
    for idx, row in coords.iterrows():
        for loci in row["locis"]:
            TAD = {}
            for cond in tad_samples:
                TAD["TAD_level_" + cond] = _get_TAD_level_final(row, cond, loci)
            up_df = {"loci":loci}
            up_df.update(TAD)
            df = df.append(row[genes_df.columns.tolist()].append(pd.Series(up_df)), ignore_index = True)
    return df

In [102]:
genes_dfs_TADs = {}
for disease in snp_names:
    genes_dfs_TADs[disease] = identify_tad_level(genes_dfs[disease], snp_dfs[disease],)

In [105]:
sample_with_replicates

['hacat_stim', 'hacat_unstim', 'myla', 'naive_T']

## generate nice looking table

In [124]:
# name your files with FULL_NAME and if you have replicates use FULL_NAME_N with N = 1,2...
names = ["hacat_stim_1","hacat_stim_2","hacat_unstim_1","hacat_unstim_2","myla_1","myla_2","naive_T_1","naive_T_2","GM12878","hacat_stim","hacat_unstim","myla","naive_T",]
# give the names of the full(combined samples)
full_names = ["GM12878","hacat_stim","hacat_unstim","myla","naive_T"]
tad_samples = ["hacat_stim","hacat_unstim","myla"]



def create_result_row(df):
    def _test_replicated(row,cond):
        reps = [x for x in names if cond in x]
        presences = [not np.isnan(row[x]) for x in reps]
        if all(presences):
            return "all"
        elif not any(presences):
            return False
        else:
            idx = np.where(presences)[0]
            return f"{', '.join(np.array(reps)[idx])} only"
        print("missed")
        return "missed"

    def _tad_string(tad):
        if tad in [1.0,2.0,3.0,4.0,5.0]:
            return int(tad)
        elif tad == 0.0:
            return "outside tad"
        return "not found"

    def _overlapping(row):
        if (row[full_names] == 1).any():
            return " (overlapping)"
        else:
            return ""
        
    sample_with_replicates = [x for x in full_names if any(x + "_" in s for s in names)]

    genes_all = {key:[] for key in ["combined"] + full_names}
    for idx, row in df.iterrows():
        genes_all["combined"].append(f"{row['gene_name']}{_overlapping(row)}")
        for condition in sample_with_replicates:
            replicated = _test_replicated(row,condition)
            if replicated:
                genes_all[condition].append(f"{row['gene_name']} ({replicated})")
                if condition in tad_samples:
                    genes_all[condition][-1] = genes_all[condition][-1] + (f", TAD level:{_tad_string(row['TAD_level_' + condition])}")
        for condition in list(set(full_names) - set(sample_with_replicates)):
            if not np.isnan(row[condition]): 
                genes_all[condition].append(f"{row['gene_name']}")
                if condition in tad_samples:
                    genes_all[condition][-1] = genes_all[condition][-1] + (f", TAD level:{_tad_string(row['TAD_level_' + condition])}")
    for key in genes_all.keys():
        genes_all[key] = ["; ".join(genes_all[key])]
    return pd.DataFrame(genes_all)


genes_dfs_TADs["PsA"].groupby("loci").apply(create_result_row).reset_index(level=-1, drop=True)


,combined,GM12878,hacat_stim,hacat_unstim,myla,naive_T
loci,,,,,,
rs12044149,SLC35D1; GADD45A; PDE4B; MIER1,SLC35D1,,,,SLC35D1 (all); GADD45A (naive_T_2 only); PDE4B...
rs12884468,PPP2R3C; KIAA0391; PSMA6,,"PPP2R3C (hacat_stim_2,hacat_stim only), TAD le...","PPP2R3C (hacat_unstim_2,hacat_unstim only), TA...",,
rs2020854,CS (overlapping); ERBB3; BAZ2A; IL23A; ATP5F1B...,CS; BAZ2A; IL23A; ATP5F1B; PTGES3; TIMELESS; G...,"CS (all), TAD level:3; BAZ2A (hacat_stim_2 onl...","CS (all), TAD level:3; TIMELESS (hacat_unstim_...","CS (all), TAD level:1; IL23A (myla only), TAD ...",CS (all); ERBB3 (all); BAZ2A (all); IL23A (all...
rs33980500,REV3L; TRAF3IP2; MFSD4B,REV3L; TRAF3IP2,"REV3L (hacat_stim_2,hacat_stim only), TAD leve...","REV3L (hacat_unstim_2,hacat_unstim only), TAD ...",,
rs34725611,ICAM1; TYK2,ICAM1,,,"ICAM1 (myla_1,myla only), TAD level:1; TYK2 (m...",
rs4921482,TTC1,TTC1,"TTC1 (hacat_stim only), TAD level:1","TTC1 (hacat_unstim only), TAD level:1",,"TTC1 (naive_T_1,naive_T only)"
rs715285,P4HA2; IRF1; PDLIM4; RAPGEF6; SLC22A4; SLC22A5...,,"P4HA2 (hacat_stim only), TAD level:2; RAPGEF6 ...","IRF1 (hacat_unstim only), TAD level:1; PDLIM4 ...","IRF1 (all), TAD level:1; SLC22A5 (myla only), ...",FNIP1 (naive_T only)
rs7552167,STPG1; NIPAL3; RCAN3; SRRM1; IL22RA1; GRHL3; C...,,"STPG1 (all), TAD level:4; NIPAL3 (all), TAD le...","STPG1 (all), TAD level:3; NIPAL3 (all), TAD le...","STPG1 (all), TAD level:3; NIPAL3 (all), TAD le...",STPG1 (all); NIPAL3 (naive_T only); SRRM1 (all...
rs76956521,CD74; NDST1; RBM22; DCTN4; TNIP1 (overlapping)...,CD74; TNIP1; RPS14; ATOX1,"NDST1 (hacat_stim only), TAD level:1; TNIP1 (h...","TNIP1 (hacat_unstim_2,hacat_unstim only), TAD ...","CD74 (myla_1,myla only), TAD level:outside tad...","CD74 (all); DCTN4 (naive_T_1,naive_T only); TN..."


In [125]:
genes_per_loci = {}
for disease in snp_names:
    genes_per_loci[disease] = genes_dfs_TADs[disease].groupby("loci").apply(create_result_row).reset_index(level=-1, drop=True)

In [126]:
for key,df in genes_per_loci.items():
    df.to_csv(
        os.path.join(output_folder, "FINAL_genes_by_loci_" + key + ".csv"))


In [127]:
writer = pd.ExcelWriter(os.path.join(output_folder,'genes_by_loci_all.xlsx'), engine='xlsxwriter')
workbook=writer.book
wrap = workbook.add_format({'text_wrap': True})

proper_names = {key: value for key, value in zip(snp_names,full_names)}
for key,df in genes_per_loci.items():
    df.to_excel(writer, sheet_name = proper_names[key])
    writer.sheets[proper_names[key]].set_column('A:A', 10)
    writer.sheets[proper_names[key]].set_column('B:G', 35, wrap)

writer.save()